In [24]:
from keras.models import Sequential, Model, load_model
from keras.utils.np_utils import to_categorical
from keras.layers import Flatten, Dense, Dropout, Reshape, Permute, Activation, BatchNormalization, \
    Input
from keras.engine.topology import merge
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback
import numpy as np
from keras.wrappers.scikit_learn import KerasRegressor
from keras.applications.resnet50 import ResNet50
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# from image_util import crop_center
import pickle as pkl
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# load data
Xtrain = np.load('./data/dataset256/Xtrain.npy')
Xtest = np.load('./data/dataset256/Xtest.npy')
Ytrain_reg = np.load('./data/dataset256/Ytrain_reg.npy')
Ytest_reg = np.load('./data/dataset256/Ytest_reg.npy')
Ytrain_cls = np.load('./data/dataset256/Ytrain_cls.npy')
Ytest_cls = np.load('./data/dataset256/Ytrain_cls.npy')

In [19]:
# netowrk settings

batch_size = 64
nb_epoch = 2
folder = './myresnet-cls-training-checkpoint/'
load_path = False
n_cls = int(np.max(Ytrain_cls))


In [25]:
Ytrain_cls = to_categorical(Ytrain_cls-1)

In [13]:
def addResidualBlock(inp, size=(3,3), subsample=(4,4), filters_conv_rate=2, name=None):
    conv_filters = int(inp.get_shape()[-1])*filters_conv_rate
    inp = BatchNormalization()(inp)
    first = Convolution2D(conv_filters, size[0], size[1], border_mode='same', subsample=subsample, activation='relu')(inp)
    
    second = BatchNormalization()(first)
    second = Convolution2D(conv_filters, 3,3, border_mode='same', activation='relu')(second)
    
    second = BatchNormalization()(second)
    second = Convolution2D(conv_filters, 3,3, border_mode='same', activation='relu')(second)
    
    return merge([first, second], mode='sum', name=name)

In [20]:
def MyResNetRegr():
    inp = Input(shape=(256, 256,3))
    out = Convolution2D(16, 3,3, border_mode='same', activation='relu', subsample=(4,4))(inp)
    for i in range(3):
        out = addResidualBlock(out)
        out = BatchNormalization()(out)
        out = Activation('relu')(out)
    
    out = BatchNormalization()(out)
    out = Flatten()(out)
    out = Dense(128, activation='relu', name='fcc')(out)
    out = Dense(1, name='out')(out)
    
    model = Model(input=inp, output=out)
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def MyResNetCls():
    inp = Input(shape=(256, 256,3))
    out = Convolution2D(16, 3,3, border_mode='same', activation='relu', subsample=(4,4))(inp)
    for i in range(3):
        out = addResidualBlock(out)
        out = BatchNormalization()(out)
        out = Activation('relu')(out)
    
    out = BatchNormalization()(out)
    out = Flatten()(out)
    out = Dense(128, activation='relu', name='fcc')(out)
    out = Dense(n_cls, activation='softmax', name='out')(out)
    
    model = Model(input=inp, output=out)
    
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model


In [21]:
if load_path:
    model = load_model(load_path)
else:
    model = MyResNetCls()
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 256, 256, 3)   0                                            
____________________________________________________________________________________________________
convolution2d_21 (Convolution2D) (None, 64, 64, 16)    448         input_3[0][0]                    
____________________________________________________________________________________________________
batchnormalization_27 (BatchNorm (None, 64, 64, 16)    64          convolution2d_21[0][0]           
____________________________________________________________________________________________________
convolution2d_22 (Convolution2D) (None, 16, 16, 32)    4640        batchnormalization_27[0][0]      
___________________________________________________________________________________________

In [22]:
# batch logger 
# ref: https://github.com/fchollet/keras/issues/2850#issuecomment-222542429
class NBatchLogger(Callback):
    def __init__(self,display=100):
        '''
        display: Number of batches to wait before outputting loss
        '''
        self.seen = 0
        self.display = display

    def on_batch_end(self,batch,logs={}):
        self.seen += logs.get('size', 0)
        if self.seen % self.display == 0:
            print('\n{0}/{1} - Batch Loss: {2}'.format(self.seen,self.params['nb_sample'],
                                                self.params['metrics'][0]))

In [27]:
#train just 1000 samples, 3 epochs
# if False:
checkpoint = ModelCheckpoint(filepath=folder + 'checkpoint-{epoch:02d}-{val_loss:.2f}.hdf5')
out_batch = NBatchLogger(display=1)
history = model.fit(x=Xtrain[:1000], y=Ytrain_cls[:1000], 
          nb_epoch=nb_epoch, batch_size=batch_size,validation_split=0.1, verbose=1, callbacks=[checkpoint]) 

Train on 900 samples, validate on 100 samples
Epoch 1/2

64/900 - Batch Loss: loss
 64/900 [=>............................] - ETA: 84s - loss: 2.1175
128/900 - Batch Loss: loss
128/900 [===>..........................] - ETA: 42s - loss: 2.0771
192/900 - Batch Loss: loss
192/900 [=====>........................] - ETA: 28s - loss: 2.0610
256/900 - Batch Loss: loss
256/900 [=======>......................] - ETA: 20s - loss: 2.0391
320/900 - Batch Loss: loss
320/900 [=========>....................] - ETA: 16s - loss: 2.0312
384/900 - Batch Loss: loss
384/900 [===========>..................] - ETA: 12s - loss: 1.9975
448/900 - Batch Loss: loss
448/900 [=============>................] - ETA: 10s - loss: 1.9586
512/900 - Batch Loss: loss
512/900 [================>.............] - ETA: 8s - loss: 1.9602 
576/900 - Batch Loss: loss
576/900 [==================>...........] - ETA: 6s - loss: 1.9496
640/900 - Batch Loss: loss
640/900 [====================>.........] - ETA: 4s - loss: 1.9493
704/90

In [16]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

NameError: name 'history' is not defined

In [2]:
# validation keras fix 
# ref: http://stackoverflow.com/questions/41796618/python-keras-cross-val-score-error/41841066#41841066
from keras.wrappers.scikit_learn import BaseWrapper
import copy

def custom_get_params(self, **params):
    res = copy.deepcopy(self.sk_params)
    res.update({'build_fn': self.build_fn})
    return res

BaseWrapper.get_params = custom_get_params

In [23]:
evl = model.evaluate(x=Xtest, y=Ytest,batch_size=batch_size)

3840/3852 [============================>.] - ETA: 0s 

In [24]:
evl

0.0042826834603561896

In [20]:
evl

0.002468179222131061

In [ ]:
# regression 
# rev: http://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/

In [ ]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=MyNet, nb_epoch=nb_epoch, batch_size=batch_size, verbose=1)

In [ ]:
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, Xtrain[:1000], Ytrain[:1000], cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))